In [ ]:
✅ Palo Alto Firewall Onboarding into Splunk (Short & Crisp SOP)
1) Architecture / Data Flow

Palo Alto → Syslog Server (Linux) → Splunk Heavy Forwarder → Indexers → Search Head / ES

✅ Reason:

Firewall sends only syslog

HF used for parsing + forwarding reliability

2) Details given to Firewall Team

Provide only:

Destination IP: 192.168.4.144

Port: 66 (or 514 as per standard)

Protocol: UDP/TCP

Log types: Traffic, Threat, System, Config, UserID, GlobalProtect

3) Syslog Server Setup (Linux)
3.1 Create syslog folder
sudo mkdir -p /syslog_data/paloalto

3.2 Enable syslog receiver on port 66 (UDP)

Create:

sudo vi /etc/rsyslog.d/01-listen.conf


Add:

module(load="imudp")
input(type="imudp" port="66")

3.3 Write logs into Firewall-wise folders (based on source IP)

Create:

sudo vi /etc/rsyslog.d/20-paloalto.conf


Add:

if ($fromhost-ip == '10.10.10.5') then /syslog_data/paloalto/FW01/traffic.log
& stop

if ($fromhost-ip == '10.10.10.6') then /syslog_data/paloalto/FW02/traffic.log
& stop

3.4 Restart rsyslog
sudo systemctl restart rsyslog


✅ Output:

/syslog_data/paloalto/FW01/traffic.log
/syslog_data/paloalto/FW02/traffic.log

4) Install Splunk Heavy Forwarder (HF) on Syslog Server

HF role:

monitor syslog files

forward to Indexers

Check:

splunk status

5) Create Inputs App on Deployment Server

Create app:

/opt/splunk/etc/deployment-apps/afrexim_all_paloalto_inputs/local/inputs.conf

6) inputs.conf (HF File Monitoring)
[monitor:///syslog_data/paloalto/*/*.log]
disabled = 0
index = paloalto
sourcetype = pan:log
host_segment = 3


✅ host_segment=3 makes host = firewall folder name
Example:
/syslog_data/paloalto/FW01/traffic.log → host=FW01

7) Install Palo Alto Add-on (TA)

TA name:

Splunk_TA_paloalto

Install on:

✅ Heavy Forwarder (parsing / sourcetype routing)

✅ Search Head / SHC (field extraction + CIM)

TA provides:

Field extraction

Routing to: pan:traffic, pan:threat, pan:system

CIM compliance

8) Deployment Server – Server Class Mapping (2 Apps)
Step 1: Keep both apps in deployment server
/opt/splunk/etc/deployment-apps/
   afrexim_all_paloalto_inputs/
   Splunk_TA_paloalto/

Step 2: Create Server Class

UI path:
Settings → Forwarder Management → Server Classes → New

Server class:
afrem_all_paloalto_inputs

Step 3: Add Apps under same Server Class (Apps = 2)

afrem_all_paloalto_inputs

Splunk_TA_paloalto

Step 4: Add Clients (HFs)

SPLUNK-HFSYS01

SPLUNK-HFSYS03

SPLUNK-HFSYS04

SPLUNK-HFSYS05

Step 5: Reload DS
splunk reload deploy-server


✅ Result on HF:

$SPLUNK_HOME/etc/apps/
  afrexim_all_paloalto_inputs/
  Splunk_TA_paloalto/

9) Final Data Flow After Deployment

Firewall → Syslog port → file created → HF monitors → TA parses → HF forwards → Indexers store in index=paloalto

10) Validation (Interview must)
Check data
index=paloalto earliest=-15m

Host validation
index=paloalto earliest=-15m
| stats count by host

Sourcetype validation
index=paloalto earliest=-15m
| stats count by sourcetype


Expected:

pan:traffic

pan:threat

pan:system

CIM check (ES)
| tstats count where index=paloalto by sourcetype